# # 모듈 및 함수
---

In [1]:
import pandas as pd
import numpy as np
import json
import glob
import os
import pathlib
import re
import shutil
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from PIL import Image, ImageDraw, ImageFont


def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    x = np.array(x, dtype=float).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    y = list(y.reshape(-1))
    return y


def xyxy2xywhn(x, w=1920, h=1200, clip=False, eps=0.0):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] normalized where xy1=top-left, xy2=bottom-right
    if clip:
        clip_boxes(x, (h - eps, w - eps))  # warning: inplace clip
    x = np.array(x, dtype=float).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = ((x[:, 0] + x[:, 2]) / 2) / w  # x center
    y[:, 1] = ((x[:, 1] + x[:, 3]) / 2) / h  # y center
    y[:, 2] = (x[:, 2] - x[:, 0]) / w  # width
    y[:, 3] = (x[:, 3] - x[:, 1]) / h  # height
    y = list(y.reshape(-1))
    return y

In [15]:
extrs = []
for calib in glob.glob('/NIA50/50-2/data/nia50_all/raw/*/calib/camera/*.json'):
    with open(calib, 'r') as f:
        calib = json.load(f)

    extr = calib['extrinsic']
    if extr not in extrs:
        extrs.append(extr)

In [17]:
len(extrs)

4

# # NIA50
---

## # labels
---

In [3]:
# 2D 라벨 사용

src_path = '/NIA50/50-2/data/nia50_all/raw/'
save_path = '/NIA50/50-2/data/nia50_all/yolov5/labels/'
if os.path.isdir(save_path)==False:
    os.makedirs(save_path)

class_ls = {'Small_Car': 0,
            'Light_Car': 1,
            'Car': 2,
            'Van': 3,
            'SUV': 4,
            'Small_Truck': 5,
            'Medium_Truck': 6,
            'Large_Truck': 7,
            'Mini_Bus': 8,
            'Bus': 9,
            'Special_Vehicle': 10,
            'Two_Wheeler': 11,
            'Kickboard': 12,
            'Adult': 13,
            'Kid': 14}

scenes = sorted(os.listdir(src_path))
for scene in scenes:
    labels = os.listdir(src_path+scene+'/2d_label')

    for label in labels:
        with open(src_path+scene+'/2d_label/'+label) as f:
            label_js = json.load(f)

        frame = label_js['filename'][:4]
        div_w, div_h = label_js['width']/1920, label_js['height']/1200
        objs = label_js['objects']
    
        labeling_ls = []
        for obj in objs:
            if obj['class'] in class_ls.keys():
                cls_num = class_ls[obj['class']]
                bbox_3d = obj['front'] + obj['back']

                x = []
                y = []
                for cordi in bbox_3d:
                    x.append(cordi['x'])
                    y.append(cordi['y'])

                x = np.asarray(x, dtype=float) / div_w
                y = np.asarray(y, dtype=float) / div_h

                x[x > 1920] = 1920
                x[x < 0] = 0
                y[y > 1200] = 1200
                y[y < 0] = 0

                xmin, ymin, xmax, ymax = min(x), min(y), max(x), max(y)

                # if xmax==0 or ymax==0:
                    # continue
                
                bbox_2d = [xmin, ymin, xmax, ymax]
                bbox_2d = xyxy2xywhn(bbox_2d)               

                if bbox_2d[2]==0 or bbox_2d[3]==0:
                    continue
        
                labeling = ' '.join(map(str, [cls_num] + bbox_2d))
                labeling_ls.append(labeling)

        with open(save_path+scene+f'_{frame}'+'.txt', 'w') as f:
            f.write('\n'.join(labeling_ls))

In [2]:
# 3D 라벨을 사용해서 만들기

src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/raw/'
save_path = '/data/NIA50/50-2/data/NIA50/nia50_all/yolov5_integ/labels/'
if os.path.isdir(save_path)==False:
    os.makedirs(save_path)

def roty(t, Rx=90/180*np.pi):
    c = np.cos(t)
    s = np.sin(t)
    # return  np.array([[c, 0, s],
    #                 [0, 1, 0],
    #                 [-s, 0, c]])

    X = np.array([[1, 0, 0],
                    [0, np.cos(Rx), -np.sin(Rx)],
                    [0, np.sin(Rx), np.cos(Rx)]])

    Z = np.array([[c, -s, 0],
                    [s, c, 0],
                    [0, 0, 1]])
    
    return np.matmul(Z, X)

change_class_ls = {'Small_Car': 'Car',
                    'Light_Car': 'Car',
                    'Car': 'Car',
                    'Van': 'Van',
                    'SUV': 'SUV',
                    'Small_Truck': 'Truck',
                    'Medium_Truck': 'Truck',
                    'Large_Truck': 'Truck',
                    'Mini_Bus': 'Bus',
                    'Bus': 'Bus',
                    'Special_Vehicle': 'Special_Vehicle',
                    'Two_Wheeler': 'Two_Wheeler',
                    'Kickboard': 'Two_Wheeler',
                    'Adult': 'Person',
                    'Kid': 'Person'}

type_ls = {'Car': 0,
            'Van': 1,
            'SUV': 2,
            'Truck': 3,
            'Bus': 4,
            'Special_Vehicle': 5,
            'Two_Wheeler': 6,
            'Person': 7}

# type_ls = {'Small_Car': 0,
#         'Light_Car': 1,
#         'Car': 2,
#         'Van': 3,
#         'SUV': 4,
#         'Small_Truck': 5,
#         'Medium_Truck': 6,
#         'Large_Truck': 7,
#         'Mini_Bus': 8,
#         'Bus': 9,
#         'Special_Vehicle': 10,
#         'Two_Wheeler': 11,
#         'Kickboard': 12,
#         'Adult': 13,
#         'Kid': 14}


scenes = sorted(os.listdir(src_path))
for scene in scenes:
    try:
        with open(src_path+scene+'/calib/camera/camera_0.json') as f:
            calib = json.load(f)

        labels = os.listdir(src_path+scene+'/label')
        for label in labels:
            with open(src_path+scene+'/label/'+label) as f:
                label_js = json.load(f)

            labeling_ls = []
            for i in np.arange(len(label_js)):
                try:
                    xyz = label_js[i]['psr']['position']
                    R = roty(label_js[i]['psr']['rotation']['z'])
                    lwh = label_js[i]['psr']['scale']

                    x = xyz['x']
                    y = xyz['y']
                    z = xyz['z']
                    l = lwh['x']
                    w = lwh['y']
                    h = lwh['z']

                    x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2];
                    y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2];
                    z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2];

                    # corners_3d = np.vstack([x_corners, y_corners, z_corners, [1, 1, 1, 1, 1, 1, 1, 1]])
                    corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
                    # print corners_3d.shape
                    corners_3d[0, :] = corners_3d[0, :] + x  # x
                    corners_3d[1, :] = corners_3d[1, :] + y  # y
                    corners_3d[2, :] = corners_3d[2, :] + z  # z
                    corners_3d = np.vstack([corners_3d, [1, 1, 1, 1, 1, 1, 1, 1]])

                    extr = np.asarray(calib['extrinsic']).reshape(4, 4)
                    intr = np.hstack([np.asarray(calib['intrinsic']).reshape(3, 3), np.asarray([0, 0, 0]).reshape(-1, 1)])
                    # proj_mat = np.matmul(viewmatrix, extr)

                    point2d = np.matmul(intr, np.matmul(extr, corners_3d))

                    pointx = np.around(point2d/point2d[2])[0]
                    pointx[pointx > 1920] = 1920
                    pointx[pointx < 0] = 0
                    pointy = np.around(point2d/point2d[2])[1]
                    pointy[pointy > 1200] = 1200
                    pointy[pointy < 0] = 0
                    box = [min(pointx), min(pointy), max(pointx), max(pointy)]
                    box = xyxy2xywhn(box)


                    obj_type = label_js[i]['obj_type']
                    obj_type = change_class_ls[obj_type]
                    type_num = type_ls[obj_type]

                    labeling = ' '.join(map(str, [type_num] + box))
                    labeling_ls.append(labeling)
                except KeyError:
                    print(scene)

            with open(save_path+scene+f'_{label[:4]}'+'.txt', 'w') as f:
                f.write('\n'.join(labeling_ls))

    except:
        continue

L_2210_Suwon_A_A_C0002
L_2210_Suwon_A_A_C0002
L_2210_Suwon_A_A_C0016
L_2210_Suwon_A_A_C0048
L_2210_Suwon_A_A_C0048
L_2210_Suwon_A_A_C0048
L_2210_Suwon_A_A_C0048
L_2210_Suwon_A_A_C0063
L_2210_Suwon_A_A_C0063
L_2210_Suwon_A_A_C0063
L_2210_Suwon_A_A_C0063
L_2210_Suwon_A_A_C0063
L_2210_Suwon_A_A_C0112
L_2210_Suwon_A_A_C0145
L_2210_Suwon_A_A_C0147
L_2210_Suwon_A_A_C0166
L_2210_Suwon_A_A_C0166
L_2210_Suwon_A_A_C0169
L_2210_Suwon_A_N_C0025
L_2210_Suwon_A_N_C0025
L_2210_Suwon_B_A_C0010
L_2210_Suwon_B_A_C0133
L_2210_Suwon_B_A_C0222
L_2210_Suwon_B_A_C0226
L_2210_Suwon_B_A_C0275
L_2210_Suwon_B_A_C0307
L_2210_Suwon_B_A_C0393
L_2210_Suwon_B_A_C0417
L_2210_Suwon_B_F_C0154
L_2210_Suwon_B_N_C0058
L_2210_Suwon_B_N_C0225
L_2210_Suwon_B_N_C0347
L_2210_Suwon_B_N_C0347
L_2210_Suwon_B_N_C0347
L_2210_Suwon_B_N_C0347
L_2210_Suwon_B_N_C0347
L_2210_Suwon_B_N_C0347
L_2210_Suwon_B_N_C0347
L_2210_Suwon_B_N_C0347
L_2210_Suwon_B_N_C0349
L_2210_Suwon_B_N_C0349
L_2211_Pangyo_C_A_C0076
L_2211_Pangyo_C_A_C0076
L_2211_Pa

In [39]:
# # 3D 라벨을 사용해서 만들기

# src_path = '/data/NIA50/50-2/data/NIA50/train_1st/raw/'
# save_path = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/labels/'
# if os.path.isdir(save_path)==False:
#     os.makedirs(save_path)

# def roty(t, Rx=90/180*np.pi):
#     c = np.cos(t)
#     s = np.sin(t)
#     # return  np.array([[c, 0, s],
#     #                 [0, 1, 0],
#     #                 [-s, 0, c]])

#     X = np.array([[1, 0, 0],
#                     [0, np.cos(Rx), -np.sin(Rx)],
#                     [0, np.sin(Rx), np.cos(Rx)]])

#     Z = np.array([[c, -s, 0],
#                     [s, c, 0],
#                     [0, 0, 1]])
    
#     return np.matmul(Z, X)
    
# type_ls = {'Small_Car': 0,
#         'Light_Car': 1,
#         'Car': 2,
#         'Van': 3,
#         'SUV': 4,
#         'Small_Truck': 5,
#         'Medium_Truck': 6,
#         'Large_Truck': 7,
#         'Mini_Bus': 8,
#         'Bus': 9,
#         'Special_Vehicle': 10,
#         'Two_Wheeler': 11,
#         'Kickboard': 12,
#         'Adult': 13,
#         'Kid': 14}


# scenes = sorted(os.listdir(src_path))
# for scene in scenes:
#     try:
#         with open(src_path+scene+'/calib/camera/camera_0.json') as f:
#             calib = json.load(f)

#         labels = os.listdir(src_path+scene+'/label')
#         for label in labels:
#             with open(src_path+scene+'/label/'+label) as f:
#                 label_js = json.load(f)

#             labeling_ls = []
#             for i in np.arange(len(label_js)):
#                 try:
#                     xyz = label_js[i]['psr']['position']
#                     R = roty(label_js[i]['psr']['rotation']['z'])
#                     lwh = label_js[i]['psr']['scale']

#                     x = xyz['x']
#                     y = xyz['y']
#                     z = xyz['z']
#                     l = lwh['x']
#                     w = lwh['y']
#                     h = lwh['z']

#                     x_corners = [l / 2, l / 2, -l / 2, -l / 2, l / 2, l / 2, -l / 2, -l / 2];
#                     y_corners = [h / 2, h / 2, h / 2, h / 2, -h / 2, -h / 2, -h / 2, -h / 2];
#                     z_corners = [w / 2, -w / 2, -w / 2, w / 2, w / 2, -w / 2, -w / 2, w / 2];

#                     # corners_3d = np.vstack([x_corners, y_corners, z_corners, [1, 1, 1, 1, 1, 1, 1, 1]])
#                     corners_3d = np.dot(R, np.vstack([x_corners, y_corners, z_corners]))
#                     # print corners_3d.shape
#                     corners_3d[0, :] = corners_3d[0, :] + x  # x
#                     corners_3d[1, :] = corners_3d[1, :] + y  # y
#                     corners_3d[2, :] = corners_3d[2, :] + z  # z
#                     corners_3d = np.vstack([corners_3d, [1, 1, 1, 1, 1, 1, 1, 1]])

#                     extr = np.asarray(calib['extrinsic']).reshape(4, 4)
#                     intr = np.hstack([np.asarray(calib['intrinsic']).reshape(3, 3), np.asarray([0, 0, 0]).reshape(-1, 1)])
#                     # proj_mat = np.matmul(viewmatrix, extr)

#                     point2d = np.matmul(intr, np.matmul(extr, corners_3d))

#                     pointx = np.around(point2d/point2d[2])[0]
#                     pointx[pointx > 1920] = 1920
#                     pointx[pointx < 0] = 0
#                     pointy = np.around(point2d/point2d[2])[1]
#                     pointy[pointy > 1200] = 1200
#                     pointy[pointy < 0] = 0
#                     box = [min(pointx), min(pointy), max(pointx), max(pointy)]
#                     box = xyxy2xywhn(box)


#                     obj_type = label_js[i]['obj_type']
#                     type_num = type_ls[obj_type]

#                     labeling = ' '.join(map(str, [type_num] + box))
#                     labeling_ls.append(labeling)
#                 except KeyError:
#                     print(scene)

#             with open(save_path+scene+f'_{label[:4]}'+'.txt', 'w') as f:
#                 f.write('\n'.join(labeling_ls))

#     except:
#         print('캘리브 없음:', scene)

# del_label_ls = [i for i in glob.glob('/data/NIA50/50-2/data/NIA50/train_1st/yolov5/labels/*.txt') if '--' in i]
# for del_label in del_label_ls:
#     if os.path.isfile(del_label):
#         os.remove(del_label)

Suwon_A_2210261635_0062
Suwon_A_2210261635_0062
Suwon_A_2210261635_0062
Suwon_A_2210261635_0062
Suwon_A_2210261635_0062
Suwon_A_2210261635_0111
Suwon_A_2210261635_0144
Suwon_A_2210261635_0146
Suwon_A_2210261635_0165
Suwon_A_2210261635_0165
Suwon_A_2210261635_0168
Suwon_A_2210261926_0227
Suwon_A_2210261926_0227
Suwon_B_2210261635_0013
캘리브 없음: Suwon_B_2210261635_0175
캘리브 없음: Suwon_B_2210261635_0176
캘리브 없음: Suwon_B_2210261635_0177
캘리브 없음: Suwon_B_2210261635_0178
Suwon_B_2210261926_0260
Suwon_B_2210261926_0284
Suwon_B_2210271119_0153
Suwon_B_2210272049_0025
Suwon_B_2211010949_0067
Suwon_B_2211010949_0114
Suwon_B_2211010949_0238
캘리브 없음: Suwon_B_2211010949_0267
Suwon_B_2211010949_0317
Suwon_B_2211010949_0323
Suwon_B_2211010949_0323
캘리브 없음: Suwon_B_2211010949_0358
캘리브 없음: Suwon_B_2211010949_0359
캘리브 없음: Suwon_B_2211010949_0360
캘리브 없음: Suwon_B_2211010949_0361
캘리브 없음: Suwon_B_2211010949_0362
캘리브 없음: Suwon_B_2211010949_0363
캘리브 없음: Suwon_B_2211010949_0364
캘리브 없음: Suwon_B_2211010949_0365
캘리브 없음: 

## # images
---

In [ ]:
src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/raw/'
save_path = '/data/NIA50/50-2/data/NIA50/nia50_all/yolov5_integ/images/'
if os.path.isdir(save_path)==False:
    os.makedirs(save_path)

# del_label_ls = [i for i in glob.glob('/data/NIA50/50-2/data/NIA50/train_1st/yolov5/labels/*.txt') if '--' in i]
# for del_label in del_label_ls:
#     if os.path.isfile(del_label):
#         os.remove(del_label)

image_ls = sorted(glob.glob(src_path+'*/camera/camera_0/*.jpg'))
for image in image_ls:
    scene = re.findall('[a-zA-Z0-9_]+', image)[-5]
    frame = re.findall('[0-9]+', image)[-1]
    img_name = scene + '_' + frame + '.jpg'

    shutil.copyfile(image, save_path+img_name)

In [42]:
# src_path = '/data/NIA50/50-2/data/NIA50/train_1st/raw/'
# save_path = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/images/'
# if os.path.isdir(save_path)==False:
#     os.makedirs(save_path)

# del_label_ls = [i for i in glob.glob('/data/NIA50/50-2/data/NIA50/train_1st/yolov5/labels/*.txt') if '--' in i]
# for del_label in del_label_ls:
#     if os.path.isfile(del_label):
#         os.remove(del_label)

# image_ls = sorted(os.listdir('/data/NIA50/50-2/data/NIA50/train_1st/yolov5/labels'))
# for image in image_ls:
#     scene = image[:23]
#     frame = image[24:28]
#     img_name = scene + '_' + frame + '.jpg'

#     shutil.copyfile(src_path+scene+f'/camera/camera_0/{frame}.jpg', save_path+img_name)

## # ImageSets
---

In [ ]:
src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/raw/'
save_path = '/data/NIA50/50-2/data/NIA50/nia50_all/yolov5_integ/ImageSets/'
os.makedirs(save_path, exist_ok=True)

img_src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/yolov5_integ/images/'

# dat_typs = set([re.search('[A-Z]_[A-Z]', j).group() for j in os.listdir(src_path)])
scenes = sorted(os.listdir(src_path))
dat_typs = set([re.findall('[a-zA-Z]+_[A-Z]_[A-Z]', scene)[0] for scene in scenes])
images = sorted(glob.glob(img_src_path+'*.jpg'))

train_ls = []
val_ls = []
test_ls = []
for dat_typ in dat_typs:
    # images_typ = [image for image in images if dat_typ in image]
    scenes_typ = [scene for scene in scenes if dat_typ in scene]
    
    train_val, test = train_test_split(scenes_typ, test_size=0.2, shuffle=False, random_state=44)
    train, val = train_test_split(train_val, test_size=0.2, random_state=44)

    # train_ls += [train]
    # val_ls += val
    # test_ls += test
    for j in train:
        for image in images:
            if j in image:
                train_ls.append(image)

    for j in val:
        for image in images:
            if j in image:
                val_ls.append(image)

    for j in test:
        for image in images:
            if j in image:
                test_ls.append(image)

with open(save_path+'train.txt', 'w') as f:
    f.write('\n'.join(sorted(train_ls)))
    
with open(save_path+'val.txt', 'w') as f:
    f.write('\n'.join(sorted(val_ls)))

with open(save_path+'test.txt', 'w') as f:
    f.write('\n'.join(sorted(test_ls)))

In [50]:
# src_path = '/data/NIA50/50-2/data/NIA50/train_1st/raw/'
# save_path = '/data/NIA50/50-2/data/NIA50/train_1st/yolov5/ImageSets/'
# if os.path.isdir(save_path)==False:
#     os.makedirs(save_path)

# images = sorted(glob.glob('/data/NIA50/50-2/data/NIA50/train_1st/yolov5/images/*.jpg'))

# train, val = train_test_split(images, test_size=0.2, random_state=0)

# with open(save_path+'train.txt', 'w') as f:
#     f.write('\n'.join(train))
    
# with open(save_path+'val.txt', 'w') as f:
#     f.write('\n'.join(val))

In [18]:
# import re
# import os

# with open('/data/NIA50/50-2/data/NIA50/학습용데이터_pre4/Suwon/ImageSets/train.txt') as f:
#     train = f.readlines()


# with open('/data/NIA50/50-2/data/NIA50/학습용데이터_pre4/Suwon/ImageSets/val.txt') as f:
#     val = f.readlines()


# re_train = ['../Data/50-2/images_2d/'+re.sub('\n', '', i)+'.jpg' for i in train]
# re_val = ['../Data/50-2/images_2d/'+re.sub('\n', '', i)+'.jpg' for i in val]

In [19]:
# with open('/data/NIA50/kimgh/docker/Data/50-2/ImageSets_2d/train.txt', 'w') as f:
#     f.write('\n'.join(re_train))

# with open('/data/NIA50/kimgh/docker/Data/50-2/ImageSets_2d/val.txt', 'w') as f:
#     f.write('\n'.join(re_val))

## # test_images
---

In [ ]:
save_path = '/data/NIA50/50-2/data/NIA50/nia50_all/yolov5_integ/test_images/'
os.makedirs(save_path, exist_ok=True)

test_src_path = '/data/NIA50/50-2/data/NIA50/nia50_all/yolov5_integ/ImageSets/test.txt'

with open(test_src_path, 'r') as f:
    test_images = [j.replace('\n', '') for j in f.readlines()]

for image in test_images:
    if os.path.exists(image) == True:
        shutil.move(image, save_path)

# # NIA50_sample
---

In [ ]:
label_path = '/data/NIA50/data/2-050_sensor_sample/2d_label/'
front_labels = [word for word in os.listdir(label_path) if 'front' in word]
rear_labels = [word for word in os.listdir(label_path) if 'rear' in word]

In [ ]:
# 최신 sample
label_path = '/data/NIA50/data/sample/2d_label/'
rear_labels = os.listdir(label_path)

## # labels
---

### # front_label
---

In [ ]:
df_f2dl = pd.DataFrame(columns = ['x_min', 'y_min', 'x_max', 'y_max', 'id', 'class', 'filename'])

num = 0
for front_label in sorted(front_labels)[1:]:
    with open(label_path+front_label, 'rb') as f:
        front_js = json.load(f)
    
    for obj in front_js['objects']:
        
        x_point = []
        y_point = []
        for points in obj['front']:
            x_point.append(points['x'])
            y_point.append(points['y'])
            
        for points in obj['back']:
            x_point.append(points['x'])
            y_point.append(points['y'])
            
        x_min = min(x_point) / 1.6
        if x_min < 0: x_min = 0
        
        y_min = min(y_point) / 1.6
        if y_min < 0: y_min = 0
        
        x_max = max(x_point) / 1.6
        if x_max > 1280: x_max = 1280
        
        y_max = max(y_point) / 1.6
        if y_max > 960: y_max = 960
        
        id_ = obj['id']
        class_ = obj['class']
        filename = front_js['filename'][:-4]
        
        data = [x_min, y_min, x_max, y_max, id_, class_, filename]

        df_f2dl.loc[num] = data
        
        num+=1
        
df_f2dl['class'] = df_f2dl['class'].str.lower()
df_f2dl.loc[(df_f2dl['class'] == 'box-svg-selected') | (df_f2dl['class'] == 'medium_truck'), 'class'] = 'truck'
df_f2dl.loc[(df_f2dl['class'] == 'adult'), 'class'] = 'person'
df_f2dl.loc[(df_f2dl['class'] == 'suv'), 'class'] = 'car'
# df_f2dl['class'] = df_f2dl['class'].apply(lambda x: x.capitalize())
# df_f2dl['label'] = LabelEncoder().fit_transform(df_f2dl['class'])
df_f2dl.loc[df_f2dl['class'] == 'person', 'label'] = 0
df_f2dl.loc[df_f2dl['class'] == 'car', 'label'] = 2
df_f2dl.loc[df_f2dl['class'] == 'bus', 'label'] = 5
df_f2dl.loc[df_f2dl['class'] == 'truck', 'label'] = 7
df_f2dl['label'] = df_f2dl['label'].astype('int')

df_f2dl

In [ ]:
print(df_f2dl['class'].unique())
print(df_f2dl['label'].unique())

### # rear_label
---


In [ ]:
df_r2dl = pd.DataFrame(columns = ['x_min', 'y_min', 'x_max', 'y_max', 'id', 'class', 'filename'])

num = 0
for rear_label in sorted(rear_labels)[1:]:
    with open(label_path+rear_label, 'rb') as f:
        rear_js = json.load(f)
    
    for obj in rear_js['objects']:
        
        x_point = []
        y_point = []
        for points in obj['front']:
            x_point.append(points['x'])
            y_point.append(points['y'])
            
        for points in obj['back']:
            x_point.append(points['x'])
            y_point.append(points['y'])
            
        x_min = min(x_point) / 1.6
        if x_min < 0: x_min = 0
        
        y_min = min(y_point) / 1.6
        if y_min < 0: y_min = 0
        
        x_max = max(x_point) / 1.6
        if x_max > 1280: x_max = 1280
        
        y_max = max(y_point) / 1.6
        if y_max > 960: y_max = 960
        
        id_ = obj['id']
        class_ = obj['class']
        filename = rear_js['filename'][:-4]
        
        data = [x_min, y_min, x_max, y_max, id_, class_, filename]

        df_r2dl.loc[num] = data
        
        num+=1

df_r2dl

In [ ]:
print(df_r2dl['class'].unique())
# print(df_r2dl['label'].unique())

# # yolov5용 label 만들기
---

In [ ]:
def kitti_to_yolo(label, x1, y1, x2, y2, image_w, image_h): # x1=x_min, y1=y_min, x2=x_max, y2=y_max
    return [label, ((x2 + x1)/(2*image_w)), ((y2 + y1)/(2*image_h)), (x2 - x1)/image_w, (y2 - y1)/image_h]

In [ ]:
for filename in df_f2dl['filename'].unique():

    df_lb = pd.DataFrame(columns = ['class', 'x', 'y', 'width', 'height'])
    for i, j in enumerate(df_f2dl.loc[df_f2dl['filename'] == filename].index):
        tmp = df_f2dl.iloc[j]
        label_data = kitti_to_yolo(tmp['label'], tmp['x_min'], tmp['y_min'], tmp['x_max'], tmp['y_max'], 1280, 960)
        
        df_lb.loc[i] = label_data
        df_lb['class'] = df_lb['class'].astype('int')
        
    df_lb = df_lb.loc[df_lb['width'] > 0]
    df_lb = df_lb.loc[df_lb['height'] > 0]
    df_lb.to_csv(f'/data/NIA50/data/2-050_sensor_sample/camera/front/labels/{filename}.txt', index = None, header = None, sep = ' ')

## # front
---

In [ ]:
from PIL import Image, ImageDraw, ImageFont

font_size = 15
color = (0, 255, 0)

for j in df_f2dl['filename'].unique()[:1]:
    img = f'/data/NIA50/data/2-050_sensor_sample/camera/front/images/{j}.jpg'
    img_ = Image.open(img).convert('RGB')
    
    temp = df_f2dl.loc[df_f2dl['filename'] == j]
    for i in temp.index:
        data = temp.loc[i]
        
        # color = tuple(np.random.choice(255, 3))
        font = ImageFont.truetype('/data/NIA50/kimgh/fonts/arial.ttf', 18) # arial.ttf 글씨체, font_size=15
        # font = ImageFont.load_default()
        # box_color_RGBA  = (0,255,0,255)
        # fill_color_RGBA = (0,255,0,50)
        # draw = ImageDraw.Draw(img_, 'RGBA') # RGBA
        # draw.rectangle((100,100,300,300), outline=box_color_RGBA, fill=fill_color_RGBA, width = 3)
        # img_.show()

        draw = ImageDraw.Draw(img_)
        text_pos = (data[0], data[1]-23)
        draw.text(text_pos, f"{data['id'], data['class']}", fill = color, font = font, stroke_width = 1)
        draw.rectangle(list(data[:4]), outline = color, width = 3)
    
    print('filename :', j)    
    img_.show()

## # rear
---

In [ ]:
from PIL import Image, ImageDraw, ImageFont

font_size = 15
color = (0, 255, 0)

for j in df_r2dl['filename'].unique()[90:101]:
    img = f'/data/NIA50/data/sample/camera/rear/{j}.jpg'
    img_ = Image.open(img).convert('RGB')
    
    temp = df_r2dl.loc[df_r2dl['filename'] == j]
    for i in temp.index:
        data = temp.loc[i]
        
        # color = tuple(np.random.choice(255, 3))
        font = ImageFont.truetype('/data/NIA50/kimgh/fonts/arial.ttf', 18) # arial.ttf 글씨체, font_size=15
        # font = ImageFont.load_default()
        # box_color_RGBA  = (0,255,0,255)
        # fill_color_RGBA = (0,255,0,50)
        # draw = ImageDraw.Draw(img_, 'RGBA') # RGBA
        # draw.rectangle((100,100,300,300), outline=box_color_RGBA, fill=fill_color_RGBA, width = 3)
        # img_.show()

        draw = ImageDraw.Draw(img_)
        text_pos = (data[0], data[1]-23)
        draw.text(text_pos, f"{data['id'], data['class']}", fill = color, font = font, stroke_width = 1)
        draw.rectangle(list(data[:4]), outline = color, width = 3)
        
    print('filename :', j)    
    img_.show()

# # NIA48
---

## # train, val, test
---

In [ ]:
with open('/data/NIA50/50-2/data/NIA48/temp_data/ImageSets/train.txt', 'r') as f:
    train = [i.replace('\n', '') for i in f.readlines()]
    
with open('/data/NIA50/50-2/data/NIA48/temp_data/ImageSets/val.txt', 'r') as f:
    val = [i.replace('\n', '') for i in f.readlines()]

In [ ]:
# import glob
# from sklearn.model_selection import train_test_split

# img_list = sorted(glob.glob('/data/NIA50/data/yolo_aivill48/images/*.png'))

# train_list, val_list = train_test_split(img_list, test_size = 0.2, random_state = 0)

# with open('/data/NIA50/data/yolo_aivill48/train.txt', 'w') as f:
#     f.write('\n'.join(train_list) + '\n')
    
# with open('/data/NIA50/data/yolo_aivill48/val.txt', 'w') as f:
#     f.write('\n'.join(val_list) + '\n')

## # images
---

In [50]:
save_dir = '/data/NIA50/50-2/data/NIA48/temp_data/ImageSets/'
os.makedirs(save_dir, exist_ok=True)

image_path = '/data/NIA50/50-2/data/NIA48/temp_data/images/'

train_path = [(image_path + i + '.png') for i in train]
with open(save_dir + 'train_yolo.txt', 'w') as f:
    f.write('\n'.join(train_path))
    
val_path = [(image_path + i + '.png') for i in val]
with open(save_dir + 'val_yolo.txt', 'w') as f:
    f.write('\n'.join(val_path))

In [ ]:
# # 이미지 복사

# import os
# import shutil


# path = '/data/NIA50/data/yolo_aivill48/'

# for i in avante_labels:
#     os.makedirs(path+i, exist_ok=True)
 
    
# all_path = '/data/NIA50/data/Avante_Clips/*/*'

# paths = []
# for i in glob.glob(all_path):
#     for j in avante_labels:
#         if j in i:
#             paths.append(i)
            

# # 폴더 생성
# for path in paths:
#     shutil.copytree(path + '/Camera/CameraFront/blur', '/data/NIA50/data/yolo_aivill48/' + path[-15:], dirs_exist_ok=True)

## # labels 
---

In [14]:
def xywh2xyxy(x):
    # Convert nx4 boxes from [x, y, w, h] to [x1, y1, x2, y2] where xy1=top-left, xy2=bottom-right
    x = np.array(x).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2  # top left x
    y[:, 1] = x[:, 1] - x[:, 3] / 2  # top left y
    y[:, 2] = x[:, 0] + x[:, 2] / 2  # bottom right x
    y[:, 3] = x[:, 1] + x[:, 3] / 2  # bottom right y
    y = list(y.reshape(-1))
    return y


def xyxy2xywhn(x, w=1920, h=1200, clip=False, eps=0.0):
    # Convert nx4 boxes from [x1, y1, x2, y2] to [x, y, w, h] normalized where xy1=top-left, xy2=bottom-right
    if clip:
        clip_boxes(x, (h - eps, w - eps))  # warning: inplace clip
    x = np.array(x).reshape(1, -1)
    y = np.copy(x)
    y[:, 0] = ((x[:, 0] + x[:, 2]) / 2) / w  # x center
    y[:, 1] = ((x[:, 1] + x[:, 3]) / 2) / h  # y center
    y[:, 2] = (x[:, 2] - x[:, 0]) / w  # width
    y[:, 3] = (x[:, 3] - x[:, 1]) / h  # height
    y = list(y.reshape(-1))
    return y

In [35]:
import json
import re

save_dir = '/data/NIA50/50-2/data/NIA48/temp_data/labels_yolo/'
os.makedirs(save_dir, exist_ok=True)

# {MOTORCYCLE: [BICYCLE, MOTORCYCLE], CAR: [CAR, BUS, TRUCK, ETC], PEDESTRIAN: PEDESTRIAN}
class_ = {'CAR': 0, 'BUS': 0, 'TRUCK': 0, 'ETC': 0, 'BIBYCLE': 1, 'MOTORCYCLE': 1, 'PEDESTRIAN': 2}

label_path = '/data/NIA50/50-2/data/NIA48/temp_data/labels_json/'
# labels = glob.glob(label_path+'*.json')
labels = sorted(os.listdir(label_path))

for label in labels:                     
    with open(label_path+label, 'r') as f:
        label_js = json.load(f)
        name = re.sub('FC_', '', label[:-5])
        f.close()
        
        with open (save_dir + name + '.txt', 'w') as f:
            
            for i in np.arange(len(label_js['annotation'])):
                category = label_js['annotation'][i]['category']
                xywh = label_js['annotation'][i]['3d_box'][0]['2d_box']
                
                if xywh[0] < 0: xywh[0] = 0
                
                f.write(f"{class_[category]} {xywh[0]/1920} {xywh[1]/1200} {xywh[2]/1920} {xywh[3]/1200}\n")

            f.close()

In [ ]:
# path = '/data/NIA50/data/Avante_Clips/'
# avante_clip = []
# for i in os.listdir(path):
#     clip = os.listdir(path+i)
#     avante_clip+=clip

In [ ]:
# # 라벨 복사

# import glob
# import shutil
    
# sorted(avante_clip)
# all_path = glob.glob('/data/NIA50/data/aivill-48/*/*/label')

# avante_labels = []
# for path in all_path:
#     for i in os.listdir(path):
#         if 'A_Clip' in i:
#             # print(path + '/' + i)
#             shutil.copytree(path + '/' + i + '/result', '/data/NIA50/data/yolo_aivill48/' + i, dirs_exist_ok=True)
#             avante_labels.append(i)

# # 특수환경 자율주행 3D 이미지
---

In [ ]:
val_df = pd.read_csv('/data/NIA50/data/특수환경 자율주행 3D 이미지/val_df.csv')
val_df['frame'] = val_df['frame'].apply(lambda x: str(x).zfill(6))
tdf = val_df.loc[val_df['xyzlwh']!='[0.0, 0.0, 0.0, 0, 0, 0]']
tdf.head()

In [ ]:
tdf['class'].unique()

In [ ]:
temp = t_df.loc[t_df['xyxy'].apply(lambda x: '-' in x)]

In [ ]:
tt = json.loads(temp['xyxy'].values[0])

In [ ]:
img_ = Image.open('/data/NIA50/data/특수환경 자율주행 3D 이미지/Validation/003/1/drive_0009/image_0/daejeon_003_1_drive_0009_00000034.jpg').convert('RGB')


color = tuple(np.random.choice(255, 3))
font = ImageFont.truetype('/data/NIA50/kimgh/fonts/arial.ttf', 18) # arial.ttf 글씨체, font_size=15
# font = ImageFont.load_default()
# box_color_RGBA  = (0,255,0,255)
# fill_color_RGBA = (0,255,0,50)
# draw = ImageDraw.Draw(img_, 'RGBA') # RGBA
# draw.rectangle((100,100,300,300), outline=box_color_RGBA, fill=fill_color_RGBA, width = 3)
# img_.show()

draw = ImageDraw.Draw(img_)
# text_pos = (data[0], data[1]-23)
# draw.text(text_pos, f"{data['id'], data['class']}", fill = color, font = font, stroke_width = 1)
draw.rectangle(js['OBJECT_LIST'][0]['3D_LIST'][0]['BOX'], outline = color, width = 3)
img_
